<a href="https://colab.research.google.com/github/Ndeffoloic/RAGImplementation/blob/main/RobKerrAIGenerativeChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pinecone
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [4]:
#Base python data handling environment import
import pandas as pd
import os
from tqdm.auto import tqdm
import time


#pinecode is a cloud-based vector Database we will use to store embedding
import pinecone

#OpenAI is used for the embedding LLM and GenAI model used to generate responses.
import openai

#Langchain is middleware that ties together the components of the embedding and retrieval pipelines
#The embedding chain creates searchable vectors of our data

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

#A link in the chain to operate a chat session
from langchain.chat_models import ChatOpenAI

#we will maintain some memory of the chat so follow-up questions will be context-sensitive
from langchain.chains.conversation.memory \
import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# Get Environment Variables

In [5]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY
EMBEDDING_MODEL="text-embedding-ada-002"
GENAI_MODEL='gpt-3.5-turbo'

PINECONE_KEY=os.getenv("PINECONE_KEY")
PINECONE_ENV=os.getenv("PINECONE_ENV")
PINECONE_INDEX_NAME="default" #this will be created below.

#Read Input Data

In [6]:
URL="https://rhkdemo.blob.core.windows.net/demodata/squad-content.tsv"
df = pd.read_csv(URL, sep='\t')
df.head()


URLError: <urlopen error [Errno -2] Name or service not known>

In [ ]:
df.shape

In [3]:
#Fetch only context knowledge about Detroit and London
filtered_df = df.loc[df['subject'].isin(['Detroit','London'])]
print(filtered_df['Subject'].value_counts())
filtered_df.head()


NameError: name 'df' is not defined